In [1]:
import ee

# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [2]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [3]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            100.67799289874142,
            6.339057689337892
          ],
          [
            100.67596196772081,
            6.3382887311459655
          ],
          [
            100.67522051671278,
            6.336943051553064
          ],
          [
            100.67264155668596,
            6.326561976772766
          ],
          [
            100.6766711817279,
            6.325216266618256
          ],
          [
            100.67841197974587,
            6.326369732679765
          ],
          [
            100.68079751777259,
            6.326241569910792
          ],
          [
            100.68086199177264,
            6.327010546046168
          ],
          [
            100.68150673177979,
            6.327491155548643
          ],
          [
            100.68131330977798,
            6.330374803178657
          ],
          [
            100.68095870277352,
            6.331047651975837
          ],
          [
            100.68150673177979,
            6.332201105017177
          ],
          [
            100.68260278979051,
            6.331496217353248
          ],
          [
            100.68366661080216,
            6.331303975096517
          ],
          [
            100.684601483812,
            6.331848661305088
          ],
          [
            100.6842468768092,
            6.332681709689936
          ],
          [
            100.68518174981904,
            6.3354051277208185
          ],
          [
            100.68421463980837,
            6.338160571336402
          ],
          [
            100.6780251357423,
            6.339057689337892
          ]
        ],
        "type": "LineString"
      }
    }
  ]
}

In [4]:
coords = geoJSON['features'][0]\
                ['geometry']\
                ['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [5]:
collection = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
              .filterBounds(aoi) \
              .filterDate('2020-01-01', '2020-12-31') \
              .first()\
              .clip(aoi))

In [6]:
collection.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60']

In [7]:
import IPython.display as disp

url = collection.select('B12').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [8]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(collection.select('B8'),
                   collection.select('B12'),
                   collection.select('B11').divide(collection.select('B8')))


# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [0, -20, 0], 'max': [0, 1, 1]}, 'NDVI Overlay')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [9]:
type(m)

folium.folium.Map

In [10]:
import pandas as pd



In [11]:
import numpy as np

# Load a Sentinel-2 surface reflectance image collection and filter by location and date, and select the B8 band
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
              .filterBounds(aoi) \
              .filterDate('2020-01-01', '2020-12-31') \
              .select('B8','B11','B12')

# Get the first image in the collection and clip it to the AOI
image = collection.first().clip(aoi)

# Get the mean B8 value for all the pixels within the AOI
b8_value = image.select('B8').reduceRegion(ee.Reducer.toList(), aoi, 30).get('B8')
b11_value = image.select('B11').reduceRegion(ee.Reducer.toList(), aoi, 30).get('B11')
b12_value = image.select('B12').reduceRegion(ee.Reducer.toList(), aoi, 30).get('B12')

NDVI8 = np.array(b8_value.getInfo())/8000
NDVI11 = np.array(b11_value.getInfo())/8000
NDVI12 = np.array(b12_value.getInfo())/8000  
# # Print the mean B8 value
# print('Mean B8 value: ', b8_value.getInfo())

In [12]:
NDVI12

array([0.209   , 0.196375, 0.1775  , 0.121375, 0.10075 , 0.099625,
       0.0995  , 0.103125, 0.1095  , 0.121125, 0.126375, 0.125875,
       0.12075 , 0.076125, 0.035125, 0.0335  , 0.031125, 0.03025 ,
       0.03275 , 0.028375, 0.036   , 0.0475  , 0.082625, 0.134   ,
       0.19175 , 0.2035  , 0.222   , 0.200625, 0.146875, 0.19775 ,
       0.18025 , 0.21875 , 0.13475 , 0.11275 , 0.10775 , 0.093125,
       0.093   , 0.09825 , 0.101   , 0.097   , 0.115   , 0.12125 ,
       0.10725 , 0.0365  , 0.03425 , 0.03025 , 0.030875, 0.0295  ,
       0.025875, 0.02725 , 0.03175 , 0.0585  , 0.08025 , 0.152375,
       0.13375 , 0.1255  , 0.114625, 0.132   , 0.208375, 0.182375,
       0.183125, 0.13425 , 0.120375, 0.112625, 0.09625 , 0.09975 ,
       0.0975  , 0.10125 , 0.094625, 0.11125 , 0.111   , 0.11    ,
       0.04875 , 0.03575 , 0.030875, 0.032   , 0.028875, 0.026125,
       0.02725 , 0.02875 , 0.051625, 0.06325 , 0.0885  , 0.102   ,
       0.096   , 0.089   , 0.127875, 0.2705  , 0.251125, 0.165

In [24]:
LS9 = ee.Image(ee.ImageCollection('LANDSAT/LC09/C02/T2_L2') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-01-1'), ee.Date('2023-03-17')) 
                       .first() 
                       .clip(aoi))

In [25]:
LS9.bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [26]:
import IPython.display as disp

url = LS9.select('SR_B5').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [31]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(LS9.select('SR_B5'),
                   LS9.select('SR_B6'),
                   LS9.select('SR_B1').divide(LS9.select('SR_B5')))


# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [0, -20, 0], 'max': [0, 0, 1]}, 'NDVI Overlay')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)